Data: Download the "Alzheimer's Disease.xlsx" dataset, located [here](https://osf.io/dsy37/) under "Dataset"-->"Observational Epidemiological Studies"-->"Primary Prevention". Download the dataset into the same folder as this code. Rename the dataset "AlzheimersDisease.xlsx" for better naming conventions. Later, we will also use this datset as a csv file, so you will need to also export the file as a csv.

In [ ]:
!pip install frictionless

We'll start by describing the dataset to create a resource descriptor (this file contains metadata + schema).

In [3]:
from frictionless import describe, resource # import these modules
from pprint import pprint # pretty print

# describe is the function that reads in data and automatically infers metadata & a schema
resource = describe('AlzheimersDisease.xlsx')
# We can directly use the Excel file

In [4]:
#we'll print the resource to view it
pprint(resource)

{'encoding': 'utf-8',
 'format': 'xlsx',
 'hashing': 'md5',
 'name': 'alzheimersdisease',
 'path': 'AlzheimersDisease.xlsx',
 'profile': 'tabular-data-resource',
 'schema': {'fields': [{'name': 'Outcome Being Extracted:', 'type': 'string'},
                       {'name': "Alzheimer's disease", 'type': 'integer'},
                       {'name': 'field3', 'type': 'string'},
                       {'name': 'field4', 'type': 'string'},
                       {'name': 'field5', 'type': 'string'},
                       {'name': 'field6', 'type': 'integer'},
                       {'name': 'field7', 'type': 'string'},
                       {'name': 'field8', 'type': 'string'},
                       {'name': 'field9', 'type': 'string'},
                       {'name': 'field10', 'type': 'string'},
                       {'name': 'field11', 'type': 'string'},
                       {'name': 'field12', 'type': 'integer'},
                       {'name': 'field13', 'type': 'integer'},
      

Take a look at the schema. What do you see?

The header names seem messed up. This is because the first few rows of data are empty. Let's use Layout to tell Frictionless that the header is row 3. (Note that we switch to the csv file here for simplicity)

In [6]:
from frictionless import Resource, Layout

layout = Layout(header_rows=[3])
with Resource('AlzheimersDisease.csv', layout=layout) as resource:
    pprint(resource.header)
    pprint(resource)
# more info here https://framework.frictionlessdata.io/docs/guides/framework/layout-guide/#header

['AuthorOfPaper',
 'YearOfPublication',
 'CountryOfOrigin',
 'StudyDesign',
 'PhaseOfPrevention',
 'AgeOfParticipants',
 'MenopausalStatus',
 'TimingOfHRT',
 'HRTDefinition',
 'RouteOfAdministration',
 'LengthOfFollowUp',
 'NumberOfCases',
 'NumberOfPopulation',
 'TypeOfEffectEstimate',
 'PointEstimate',
 'Lower95%CI',
 'Upper95%CI']
{'dialect': {'quoteChar': '"'},
 'encoding': 'cp1252',
 'format': 'csv',
 'hashing': 'md5',
 'layout': {'headerRows': [3]},
 'name': 'alzheimersdisease',
 'path': 'AlzheimersDisease.csv',
 'profile': 'tabular-data-resource',
 'schema': {'fields': [{'name': 'AuthorOfPaper', 'type': 'string'},
                       {'name': 'YearOfPublication', 'type': 'any'},
                       {'name': 'CountryOfOrigin', 'type': 'any'},
                       {'name': 'StudyDesign', 'type': 'any'},
                       {'name': 'PhaseOfPrevention', 'type': 'any'},
                       {'name': 'AgeOfParticipants', 'type': 'any'},
                       {'name': 'M

Take a look at the schema -- how does it look now?

Now we can save the resource as a yaml file.

In [7]:
resource.to_yaml("resource.yaml")

'path: AlzheimersDisease.csv\nlayout:\n  headerRows:\n    - 3\nname: alzheimersdisease\nprofile: tabular-data-resource\nscheme: file\nformat: csv\nhashing: md5\nstats:\n  hash: d0249d74405bce65b7b3318ac9fd9f05\n  bytes: 3167\n  fields: 17\n  rows: 0\nencoding: cp1252\ndialect:\n  quoteChar: \'"\'\nschema:\n  fields:\n    - type: string\n      name: AuthorOfPaper\n    - name: YearOfPublication\n      type: any\n    - name: CountryOfOrigin\n      type: any\n    - name: StudyDesign\n      type: any\n    - name: PhaseOfPrevention\n      type: any\n    - name: AgeOfParticipants\n      type: any\n    - name: MenopausalStatus\n      type: any\n    - name: TimingOfHRT\n      type: any\n    - name: HRTDefinition\n      type: any\n    - name: RouteOfAdministration\n      type: any\n    - name: LengthOfFollowUp\n      type: any\n    - name: NumberOfCases\n      type: any\n    - name: NumberOfPopulation\n      type: any\n    - name: TypeOfEffectEstimate\n      type: any\n    - name: PointEstimate\

Look at the schema + the data file. Is there anything in the schema you think need to be changed?

(did you look at it? if so, read on..)

I think the column 'LengthOfFollowUp' should be an integer instead of a string, but we would need to remove the 'y' from each value. Let's use Transform to do that. First we will split the field into 2, then we'll remove the new unit column we just created, then we'll update the data type to a number.

I would add a description to the metadata that says "The unit of time is in years".

In [9]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = resource
target = transform(
    source,
    steps=[
        steps.field_split(name="LengthOfFollowUp", to_names=["LengthOfFollowUp", "UnitLengthOfFollowUp"], pattern=r"\s"),
        steps.field_remove(names=["UnitLengthOfFollowUp"]),
        steps.field_update(name="LengthOfFollowUp", type="number"),
        
    ]
)
print(target.schema)
print(target.to_view())
# more info about transform here: https://framework.frictionlessdata.io/docs/guides/transform-guide

{'fields': [{'type': 'string', 'name': 'AuthorOfPaper'}, {'name': 'YearOfPublication', 'type': 'any'}, {'name': 'CountryOfOrigin', 'type': 'any'}, {'name': 'StudyDesign', 'type': 'any'}, {'name': 'PhaseOfPrevention', 'type': 'any'}, {'name': 'AgeOfParticipants', 'type': 'any'}, {'name': 'MenopausalStatus', 'type': 'any'}, {'name': 'TimingOfHRT', 'type': 'any'}, {'name': 'HRTDefinition', 'type': 'any'}, {'name': 'RouteOfAdministration', 'type': 'any'}, {'name': 'NumberOfCases', 'type': 'any'}, {'name': 'NumberOfPopulation', 'type': 'any'}, {'name': 'TypeOfEffectEstimate', 'type': 'any'}, {'name': 'PointEstimate', 'type': 'any'}, {'name': 'Lower95%CI', 'type': 'any'}, {'name': 'Upper95%CI', 'type': 'any'}, {'name': 'LengthOfFollowUp', 'type': 'number'}]}
+---------------+-------------------+-----------------+----------------+-------------------+-------------------+-------------------------+-------------+---------------+-----------------------+---------------+--------------------+--------

Note: when I saved my excel file to csv, the formatting got weird, so all my data types are now 'any'. We can update that using Transform like you saw earlier...

We can now save our newly transformed data file using write.

In [10]:
target.write('ADtransformed.csv')

{'path': 'ADtransformed.csv'}

Now you can view your saved data file! Let's also create & save a new resource descriptor for our new data file.

In [11]:
new_resource=Resource(target)

In [12]:
new_resource.to_yaml("new_resource.yaml")

"name: alzheimersdisease\nprofile: tabular-data-resource\nscheme: ''\nformat: inline\nschema:\n  fields:\n    - type: string\n      name: AuthorOfPaper\n    - name: YearOfPublication\n      type: any\n    - name: CountryOfOrigin\n      type: any\n    - name: StudyDesign\n      type: any\n    - name: PhaseOfPrevention\n      type: any\n    - name: AgeOfParticipants\n      type: any\n    - name: MenopausalStatus\n      type: any\n    - name: TimingOfHRT\n      type: any\n    - name: HRTDefinition\n      type: any\n    - name: RouteOfAdministration\n      type: any\n    - name: NumberOfCases\n      type: any\n    - name: NumberOfPopulation\n      type: any\n    - name: TypeOfEffectEstimate\n      type: any\n    - name: PointEstimate\n      type: any\n    - name: Lower95%CI\n      type: any\n    - name: Upper95%CI\n      type: any\n    - name: LengthOfFollowUp\n      type: number\n"

OK now that we've cleaned up the data some, let's add some detail to the metadata.

In [14]:
new_resource.schema.get_field('LengthOfFollowUp')
# more info: https://framework.frictionlessdata.io/docs/guides/describing-data#describing-a-schema

{'name': 'LengthOfFollowUp', 'type': 'number'}

In [15]:
new_resource.schema.get_field('LengthOfFollowUp').description = "The unit is Years"

In [16]:
pprint(new_resource.schema)

{'fields': [{'name': 'AuthorOfPaper', 'type': 'string'},
            {'name': 'YearOfPublication', 'type': 'any'},
            {'name': 'CountryOfOrigin', 'type': 'any'},
            {'name': 'StudyDesign', 'type': 'any'},
            {'name': 'PhaseOfPrevention', 'type': 'any'},
            {'name': 'AgeOfParticipants', 'type': 'any'},
            {'name': 'MenopausalStatus', 'type': 'any'},
            {'name': 'TimingOfHRT', 'type': 'any'},
            {'name': 'HRTDefinition', 'type': 'any'},
            {'name': 'RouteOfAdministration', 'type': 'any'},
            {'name': 'NumberOfCases', 'type': 'any'},
            {'name': 'NumberOfPopulation', 'type': 'any'},
            {'name': 'TypeOfEffectEstimate', 'type': 'any'},
            {'name': 'PointEstimate', 'type': 'any'},
            {'name': 'Lower95%CI', 'type': 'any'},
            {'name': 'Upper95%CI', 'type': 'any'},
            {'description': 'The unit is Years',
             'name': 'LengthOfFollowUp',
             'ty

What else do you want to change/update in the metadata or schema? Use the Describe guide to play around with the code. https://framework.frictionlessdata.io/docs/guides/describing-data

OK Let's Validate the data now!

In [18]:
from frictionless import validate

# create a report variable to store the validation report 
report = validate(new_resource)
# https://framework.frictionlessdata.io/docs/guides/validation-guide

In [19]:
# take a look at the validation report
pprint(report)

{'errors': [],
 'stats': {'errors': 9, 'tasks': 1},
 'tasks': [{'errors': [{'cell': 'NA',
                        'cells': ['Brenner',
                                  '1994',
                                  'USA',
                                  'Case-control',
                                  'Primary',
                                  '78',
                                  'NG',
                                  'Ever',
                                  'Estrogen',
                                  'Any',
                                  '107',
                                  '120',
                                  'OR',
                                  '1.1',
                                  '0.6',
                                  '1.8',
                                  'NA'],
                        'code': 'type-error',
                        'description': 'The value does not match the schema '
                                       'type and format for this fie

What errors are you finding?

...did you look?

It looks like we are getting errors in 'LengthOfFollowUp'. Let's manipulate the Report to make it easier to read.

In [20]:
# https://framework.frictionlessdata.io/docs/guides/validation-guide#validation-report
pprint(report.flatten(["rowPosition", "fieldPosition", "code", "message"]))

[[2,
  17,
  'type-error',
  'Type error in the cell "NA" in row "2" and field "LengthOfFollowUp" at '
  'position "17": type is "number/default"'],
 [3,
  17,
  'type-error',
  'Type error in the cell "NA" in row "3" and field "LengthOfFollowUp" at '
  'position "17": type is "number/default"'],
 [5,
  17,
  'type-error',
  'Type error in the cell "NA" in row "5" and field "LengthOfFollowUp" at '
  'position "17": type is "number/default"'],
 [8,
  17,
  'type-error',
  'Type error in the cell "NA" in row "8" and field "LengthOfFollowUp" at '
  'position "17": type is "number/default"'],
 [13,
  17,
  'type-error',
  'Type error in the cell "1â€“5" in row "13" and field "LengthOfFollowUp" at '
  'position "17": type is "number/default"'],
 [14,
  17,
  'type-error',
  'Type error in the cell "NA" in row "14" and field "LengthOfFollowUp" at '
  'position "17": type is "number/default"'],
 [16,
  17,
  'type-error',
  'Type error in the cell "NA" in row "16" and field "LengthOfFollowUp"

Now we can more easily see that these errors are all type errors, and we can see which row they are occuring in.

Why are we getting "NA" as an error? "NA" is being read-in as a string, but it should be recognized as a missing value. We can fix that!

In [21]:
new_resource.schema.missing_values = ["NA"]

In [22]:
# let's check to see if it is valid now!
report2 = validate(new_resource)
pprint(report2)

{'errors': [],
 'stats': {'errors': 5, 'tasks': 1},
 'tasks': [{'errors': [{'cell': '1â€“5',
                        'cells': ['Tang',
                                  '1996',
                                  'USA',
                                  'Cohort',
                                  'Primary',
                                  '74',
                                  'Postmenopausal',
                                  'Ever',
                                  'Estrogen',
                                  'Oral',
                                  '167',
                                  '1124',
                                  'HR',
                                  '0.50',
                                  '0.25',
                                  '0.90',
                                  '1â€“5'],
                        'code': 'type-error',
                        'description': 'The value does not match the schema '
                                       'type and forma

Nice! Those type errors are all gone now! (Note: you might see other errors though...this is from the last 2 rows in the dataset that are not really data. Take a look at them...)

Now we can add in some data constraints for fun. We can create data constraints that limit the content of the data. https://specs.frictionlessdata.io/table-schema/#constraints

We'll start by adding a constraint to the PointEstimate column. Let's say that all the values need to be less than one, so we'll add in a maximum of 1.

In [36]:
constrained_resource = describe('ADtransformed.csv')
#note that if your data types are "any" you will need to update them.
constrained_resource.schema.get_field('PointEstimate').type = "number"
constrained_resource.schema.get_field('PointEstimate').constraints["maximum"] = 1
# this means that only values of "M" are acceptable for the "sex" column

In [37]:
# save this descriptor
constrained_resource.to_yaml("constrained_resourceAD.yaml")

'path: ADtransformed.csv\nname: adtransformed\nprofile: tabular-data-resource\nscheme: file\nformat: csv\nhashing: md5\nencoding: utf-8\ndialect:\n  quoteChar: \'"\'\nschema:\n  fields:\n    - type: string\n      name: AuthorOfPaper\n    - name: YearOfPublication\n      type: any\n    - name: CountryOfOrigin\n      type: any\n    - name: StudyDesign\n      type: any\n    - name: PhaseOfPrevention\n      type: any\n    - name: AgeOfParticipants\n      type: any\n    - name: MenopausalStatus\n      type: any\n    - name: TimingOfHRT\n      type: any\n    - name: HRTDefinition\n      type: any\n    - name: RouteOfAdministration\n      type: any\n    - name: NumberOfCases\n      type: any\n    - name: NumberOfPopulation\n      type: any\n    - name: TypeOfEffectEstimate\n      type: any\n    - name: PointEstimate\n      type: number\n      constraints:\n        maximum: 1\n    - name: Lower95%CI\n      type: any\n    - name: Upper95%CI\n      type: any\n    - name: LengthOfFollowUp\n      

In [38]:
# create a new validation report
report3 = validate(constrained_resource)
pprint(report3)

{'errors': [],
 'stats': {'errors': 13, 'tasks': 1},
 'tasks': [{'errors': [{'cell': '1.1',
                        'cells': ['Brenner',
                                  '1994',
                                  'USA',
                                  'Case-control',
                                  'Primary',
                                  '78',
                                  'NG',
                                  'Ever',
                                  'Estrogen',
                                  'Any',
                                  '107',
                                  '120',
                                  'OR',
                                  '1.1',
                                  '0.6',
                                  '1.8',
                                  ''],
                        'code': 'constraint-error',
                        'description': 'A field value does not conform to a '
                                       'constraint.',
        

You can see that the there are now errors for the values that are over 1! What other constraints can you add?

Let's end by Packaging the data.

In [39]:
from frictionless import Package
package = Package(resources=Resource(path='ADtransformed.csv'), descriptor='constrained_resourceAD.yaml') 
# this package contains the data file + the descriptor file.

In [40]:
# let's look at the package
pprint(package)

{'dialect': {'quoteChar': '"'},
 'encoding': 'utf-8',
 'format': 'csv',
 'hashing': 'md5',
 'name': 'adtransformed',
 'path': 'ADtransformed.csv',
 'profile': 'tabular-data-resource',
 'resources': {'path': 'ADtransformed.csv'},
 'schema': {'fields': [{'name': 'AuthorOfPaper', 'type': 'string'},
                       {'name': 'YearOfPublication', 'type': 'any'},
                       {'name': 'CountryOfOrigin', 'type': 'any'},
                       {'name': 'StudyDesign', 'type': 'any'},
                       {'name': 'PhaseOfPrevention', 'type': 'any'},
                       {'name': 'AgeOfParticipants', 'type': 'any'},
                       {'name': 'MenopausalStatus', 'type': 'any'},
                       {'name': 'TimingOfHRT', 'type': 'any'},
                       {'name': 'HRTDefinition', 'type': 'any'},
                       {'name': 'RouteOfAdministration', 'type': 'any'},
                       {'name': 'NumberOfCases', 'type': 'any'},
                       {'name': 

In [41]:
# save the package
package.to_yaml('package.yaml')
# package.to_json('package.json') will save as JSON instead

'resources:\n  path: ADtransformed.csv\npath: ADtransformed.csv\nname: adtransformed\nprofile: tabular-data-resource\nscheme: file\nformat: csv\nhashing: md5\nencoding: utf-8\ndialect:\n  quoteChar: \'"\'\nschema:\n  fields:\n    - type: string\n      name: AuthorOfPaper\n    - name: YearOfPublication\n      type: any\n    - name: CountryOfOrigin\n      type: any\n    - name: StudyDesign\n      type: any\n    - name: PhaseOfPrevention\n      type: any\n    - name: AgeOfParticipants\n      type: any\n    - name: MenopausalStatus\n      type: any\n    - name: TimingOfHRT\n      type: any\n    - name: HRTDefinition\n      type: any\n    - name: RouteOfAdministration\n      type: any\n    - name: NumberOfCases\n      type: any\n    - name: NumberOfPopulation\n      type: any\n    - name: TypeOfEffectEstimate\n      type: any\n    - name: PointEstimate\n      type: number\n      constraints:\n        maximum: 1\n    - name: Lower95%CI\n      type: any\n    - name: Upper95%CI\n      type: an

And that's it! Now you can publish the package, which contains your data, metadata + schema!